In [1]:
# GET PARAMETERS
import os 
import sys
import torch 
# Get Parent folder : 

current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config
from plotting.plotting import error_per_station_calendar_pattern  
from examples.benchmark import local_get_args
# Init:
dataset_names = ["subway_in","subway_out"] # ['subway_in','netmob_POIs_per_station'],["subway_in","subway_out"],["subway_in","calendar"] # ["subway_in"] # ['data_bidon'] # ['METR_LA'] # ['PEMS_BAY']
dataset_for_coverage = ['subway_in','netmob_image_per_station'] #  ['data_bidon','netmob'] #  ['subway_in','netmob']  # ['METR_LA'] # ['PEMS_BAY']
model_name = 'ASTGCN' # 'STGCN', 'ASTGCN'
station = ['BEL','PAR','AMP','SAN','FLA']   # 'BON'  #'GER'
# ...

# Modif 
modification = {'epochs' : 1, #100
                'lr':5e-5,# 4e-4,
                'weight_decay':0.05,
                'dropout':0.15,
                #'set_spatial_units':  station,   
                #  
                'scheduler':None,
                #'scheduler':True,
                #'torch_scheduler_milestone': 5,
                #'torch_scheduler_gamma':0.997,
                #'torch_scheduler_lr_start_factor':1,

                'temporal_graph_transformer_encoder': False,
                #'TGE_num_layers' : 4, #2
                #'TGE_num_heads' :  1, #IMPOSSIBLE > 1 CAR DOIT DIVISER L = 7
                #'TGE_FC_hdim' :  32, #32

                #'NetMob_only_epsilon': True,    # True # False
                #'NetMob_selected_apps': ['Apple_iMessage','Web_Ads'],# ['Apple_iMessage','Web_Ads'], #,'Deezer','WhatsApp','Twitter'] #['Google_Maps']# ['Instagram','Google_Maps','Twitter']
                #'NetMob_transfer_mode' :  ['DL'], #,'UL'] # ['DL'] # ['UL'] #['DL','UL']
                #'NetMob_selected_tags' : ['station_epsilon100'],#['iris','stadium','station','university']#['park','stadium','university','station','shop','nightclub','parkings','theatre','iris','transit','public_transport']
                #'NetMob_expanded' : '', # '' # '_expanded'
                'stacked_contextual': True, # True # False

                'learnable_adj_matrix' : True,
                'graph_conv_type': 'graph_conv', # 'cheb_graph_conv', 'graph_conv'
                'learnable_adj_top_k': 10,
                'learnable_adj_embd_dim': 16, 
                
                #'vision_num_heads':6,
                #"vision_grn_out_dim":48,
                #'vision_model_name': 'VariableSelectionNetwork',
                #'vision_concatenation_early':True,   
                #'vision_concatenation_late':True,

                'compute_node_attr_with_attn' : False, # True ??
                #'adj_type':'corr'
                           }
# ...
#1038945

# Training and visu: 
args = local_get_args(model_name,
                    args_init = None,
                    dataset_names=dataset_names,
                    dataset_for_coverage=dataset_for_coverage,
                    modification = modification)

training_mode_to_visualise = ['test'] # ['test','valid','train']
(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_init = args,
                                                   station=station,modification=modification,
                                                   training_mode_to_visualise=training_mode_to_visualise)

Training and Hyper-parameter tuning with Ray is not possible
>>>>Model: ASTGCN; K_fold = 6; Loss function: MSE 
>>>>Model: ASTGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
nb subway_in invalid dates:  776
T_subway_out:  torch.Size([7392, 40])
vision_input_type POIs
vision_model_name None
subway_ds.U_valid torch.Size([940, 40, 7])
NetMob_ds.U_valid torch.Size([940, 40, 7])
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940
Invalid dates within this fold: 481
Considered Spatial-Unit:  Index(['AMP', 'BEL', 'BRO', 'COR', 'CUI', 'CUS', 'FLA', 'GOR', 'BLA', 'GRA',
       'GUI', 'GIL', 'HEN', 'HOT', 'LAE', 'MAS', 'MER', 'LUM', 'PRY', 'PER',
       'SAN', 'SAX', 'VMY', 'JEA', 'BON', 'CHA', 'VAI', 'VEN', 'MAC', 'GAR',
       'FOC', 'REP', 'GER', 'DEB', 'JAU', 'CPA', 'CRO', 'PAR', 'SOI', 'OGA'],
      dtype='object', name='COD_TRG

NameError: name 'blabla' is not defined

In [11]:
from scipy.sparse.linalg import eigs
import numpy as np 
import pandas as pd
L = (pd.DataFrame(np.random.rand(10,10))*10)


lambda_max = eigs(L.values , k=1, which='LR')[0].real

In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob
from constants.config import get_args,update_modif
from constants.paths import FOLDER_PATH,FILE_NAME,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter
from trainer import Trainer
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh


# Load config
model_name = 'STGCN' #'CNN'
dataset_names = ['subway_in','netmob']
args = get_args(model_name,dataset_names)

# Modification : 
args.K_fold = 5

args.ray = False
args.W = 0  # IMPORTANT AVEC NETMOB

args.epochs = 100
args.loss_function_type = 'MSE' # 'quantile'

# optimization:
args.mixed_precision = True

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob','subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'FeatureExtractor_ResNetInspired'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(args,folds)
K_subway_ds,args = K_fold_splitter.split_k_fold()
subway_ds = K_subway_ds[0]

ImportError: cannot import name 'match_period_coverage_with_netmob' from 'utils.utilities_DL' (/home/rrochas/prediction-validation/utils/utilities_DL.py)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [10]:
import torch

netmob_init = torch.Tensor([7392*4*263*287])
netmob_tensor = torch.Tensor([2934*4*263*287*8])

print('Init tensor: ',netmob_init)
print('Init feature vect: ',netmob_tensor)

print('\nblock1:')
sconv = torch.prod(torch.Tensor([64, 32, 131, 143, 8]))
tconv = torch.prod(torch.Tensor([64, 128, 131, 143, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)

print('\nblock2:')
sconv = torch.prod(torch.Tensor([64, 658, 65, 71, 8]))
tconv = torch.prod(torch.Tensor([64, 512, 65, 71, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)


Init tensor:  tensor([2.2318e+09])
Init feature vect:  tensor([7.0868e+09])

block1:
s_conv:  tensor(3.0692e+08) tconv:  tensor(1.2277e+09)

block2:
s_conv:  tensor(1.5548e+09) tconv:  tensor(1.2098e+09)
